In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def prd_calc(prd,Data):
    
    # Takes prd (integer) is the period were we need to find the data   #
    # Takes Data (Dataframe) is the data                                #
    # Return the average of every sector in this period based on spread #                                                         #
    dist=(Data.rind).value_counts()
    dist.index
    
    DataDay_one=[]
    for i in range(0,len(dist.index)):
        Day=Data.loc[Data['prd']==prd]
        Sector=Day.loc[Day['rind']==dist.index[i]]
        sret=Sector['sret']
        sret.to_numpy()
        avg_sret=np.average(sret)
        DataDay_one.append(avg_sret)
        
    return DataDay_one

In [3]:
def Values_from_correlation(ind1,ind2,corr):
    return corr[ind1][ind2]

In [4]:
def calculate_matricies(tick_ind,corr):
    corr_by_int=[]
    for int_i,sector_i in zip(tick_ind['int_tick'],tick_ind['rind']):
        lines=[]
        for int_j , sector_j in zip(tick_ind['int_tick'],tick_ind['rind']):
            lines.append(Values_from_correlation(sector_i,sector_j,corr))
        corr_by_int.append(lines)
    corr_by_int=pd.DataFrame(corr_by_int)
    corr_by_int.index=list(tick_ind['int_tick'])
    corr_by_int.columns=list(tick_ind['int_tick'])
    return corr_by_int

In [5]:
# import the data
Data=pd.read_csv('preproccesed.csv')
Data=Data.drop('Unnamed: 0',axis=1)

# Keep the data valuable for correlation
prd=Data.prd
int_tick=Data.int_tick
rind=Data.rind
sret=Data.sret
formal=pd.concat([int_tick,prd,rind,sret],axis=1)

In [6]:
# list of periods
periods=list((formal.loc[formal['int_tick']==int_tick[0]]).prd)

# for every period
SectPrd=[]
for i in range(0,len(periods)):
    SectPrd.append(prd_calc(periods[i],formal))

Sector_series=np.array(SectPrd) 
Sector_series=pd.DataFrame(Sector_series)

# Names of industries
Names_of_inds=list(((formal.rind).value_counts()).index)
Sector_series.columns=Names_of_inds


In [7]:
Sector_series

,non_cycl_HG,energy_HG,low_cyc_HY,basics_HG,util_HG,energy_HY,cap_g_HG,basics_HY,leis_trav_HY,auto_tech_HG,telco_HY,telco_HG,rtl_leis_HG,non_cycl_SG,auto_tech_HY,cycl_SG
0,0.000311,-0.000573,0.000614,-0.000188,-0.001045,-0.000472,2.855039e-04,-0.000982,0.002669,0.000001,0.001631,0.000914,0.001444,-0.000945,-0.000233,0.000233
1,-0.000010,-0.000544,0.000324,-0.000041,-0.001189,0.000209,-1.590850e-04,-0.000670,0.002428,-0.000529,0.001948,-0.000047,0.000712,-0.000512,0.001302,-0.000077
2,-0.000024,-0.000338,0.000399,-0.000041,-0.000710,0.000150,-1.078730e-04,-0.000390,0.001483,-0.000328,0.001235,-0.000016,0.000393,-0.000331,0.000797,-0.000396
3,-0.000168,-0.000259,-0.000148,-0.000032,-0.000561,0.000806,-1.617391e-04,0.000535,0.001213,-0.000212,0.000454,0.000273,-0.000005,-0.000139,0.000710,-0.000202
4,-0.000111,-0.000236,-0.000263,-0.000170,-0.000380,0.000019,-7.433514e-05,0.001304,0.001305,0.000061,-0.000338,0.000303,0.000151,-0.000033,0.002043,-0.000118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,-0.000086,-0.000051,0.000448,0.000001,-0.000205,0.000296,-2.045742e-05,0.000697,0.000772,-0.000117,0.000714,-0.000251,-0.000063,-0.000116,-0.000811,-0.000166
77,-0.000082,-0.000015,0.000504,0.000078,-0.000236,0.000363,3.250673e-05,0.000900,0.000863,-0.000083,0.000803,-0.000281,0.000081,-0.000159,-0.000814,-0.000183
78,-0.000103,-0.000120,0.000550,0.000263,-0.000293,0.000418,-5.609038e-07,0.001067,0.000937,-0.000141,0.000876,-0.000332,0.000773,-0.000160,-0.000816,-0.000208
79,-0.000106,-0.000005,0.000588,0.000051,-0.000116,0.000463,-2.642164e-05,0.001203,0.000997,-0.000150,0.000936,-0.000313,0.000673,-0.000151,-0.000817,-0.000030


In [8]:
# calculate correlation by window
corr_prd=[]
steps=[]
end=0
for start in range(0,len(Sector_series)):
    if end<len(Sector_series)-1:
        # for every window
        end=start+10
        step=Sector_series.loc[start:end]
        print(end)
        steps.append(step)
        step_corr=step.corr()
        # Make zeros those below zero
        step_corr[step_corr<0]=0
        # Normalize correlation map
        step_corr = np.array(step_corr)   
        step_corr=(step_corr-step_corr.min())/(step_corr.max()-step_corr.min()) 
        # Take the 1-corr
        step_corr=pd.DataFrame(step_corr)
        step_corr=1-step_corr

        step_corr.columns=Names_of_inds
        step_corr.index=Names_of_inds

        corr_prd.append(step_corr)

10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


In [9]:
sample_day=formal.loc[formal['prd']==periods[0]]
int_tick=sample_day.int_tick
rind=sample_day.rind
int_tick_ind=pd.concat([int_tick,rind],axis=1)
int_tick_ind=int_tick_ind.reset_index(drop=True)

In [10]:
time_series_corr=[]
iter_=1
for corr in corr_prd:
    print(iter_)
    time_series_corr.append(calculate_matricies(int_tick_ind,corr))
    iter_=iter_+1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71


In [11]:
use_periods=periods[10:end+1]

In [12]:
for prd,corr in zip(use_periods,time_series_corr):
    corr.to_csv(str(prd)+' rint-sret.csv')